# LangChain 实战：汽车销售聊天机器人

## 使用 GPT-4 构造销售话术数据



使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是经验丰富的汽车销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出，需要带上序号，例如：
1.
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
1.
[客户问题]
我在这个型号和那个型号之间犹豫不决，不知道该选哪个。
[销售回答]
理解您的困惑。让我们来仔细看看您的需求，来比较两种型号的性能参数以及它们各自的优势。这样，您可以依据您最关心的特点做出最好的选择。

2.
[客户问题]
这辆车的油耗怎么样？
[销售回答]
这款车的燃油效率是其一大亮点，它的油耗在同级别车型中相当优秀。这不仅能减少您的加油次数，长期来看还能为您节省相当一部分开支。

3.
[客户问题]
我需要考虑一下。
[销售回答]
我完全理解您需要时间来考虑这项重要投资。如果您有任何疑问或需要更多信息，我随时愿意协助。同时，如果您能在今天决定，我们有些特别优惠可以提供。

4.
[客户问题]
这款车安全性能怎么样？
[销售回答]
我们非常重视乘客的安全，这款车配备了最新的安全技术和先进的驾驶辅助系统，确保您在任何行驶情况下都能得到最大程度的保护。

5.
[客户问题]
我不太确定我是否有足够的预算。
[销售回答]
让我们来看看您的财务状况，并讨论我们的融资方案，包括可行的月付选项和当前的促销活动。我们的目标是为您找到既舒适又经济的解决方案。

6.
[客户问题]
这款车的维修成本高吗？
[销售回答]
这款车的整体维护成本在行业中处于较低水平。它的可靠性和延长的保修期限意味着您将享受到持久的高性能和低维修费用。

7.
[客户问题]
这款车是国产的还是进口的？
[销售回答]
这款车是国产的，但请不要让“国产”这个词误导您。它采用了许多国际标准的制造技术，并且在性能和可靠性方面可与许多进口车型相媲美。

8.
[客户问题]
我听说今天有特别的优惠？
[销售回答]
确实如此，今天我们有几项不错的优惠和促销活动。除了可观的现金折扣之外，还有其他值得考虑的福利。我们可以详细讨论哪些优惠最适合您。

9.
[客户问题]
这款车的再销售价值怎么样？
[销售回答]
很高兴您提到这一点，因为这款车的保值率是它的一大卖点。它的设计经典时尚，技术先进，因此无论现在还是几年后出售，都能保持良好的市场价值。

10.
[客户问题]
这辆车的驾驶体验如何？
[销售回答]
这款车的驾驶体验非常棒。
```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_car_sales_data.txt](real_car_sales_data.txt) 文件中

In [15]:
with open("real_car_sales_data.txt", encoding="utf-8") as f:
    real_car_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [16]:
from langchain.text_splitter import CharacterTextSplitter

In [31]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [32]:
docs = text_splitter.create_documents([real_car_sales])

Created a chunk of size 109, which is longer than the specified 100
Created a chunk of size 101, which is longer than the specified 100
Created a chunk of size 119, which is longer than the specified 100
Created a chunk of size 116, which is longer than the specified 100
Created a chunk of size 106, which is longer than the specified 100
Created a chunk of size 105, which is longer than the specified 100
Created a chunk of size 105, which is longer than the specified 100
Created a chunk of size 101, which is longer than the specified 100
Created a chunk of size 102, which is longer than the specified 100
Created a chunk of size 116, which is longer than the specified 100
Created a chunk of size 108, which is longer than the specified 100
Created a chunk of size 104, which is longer than the specified 100
Created a chunk of size 106, which is longer than the specified 100
Created a chunk of size 187, which is longer than the specified 100
Created a chunk of size 101, which is longer tha

In [37]:
docs[0]

Document(page_content='[客户问题]\n我在这个型号和那个型号之间犹豫不决，不知道该选哪个。\n[销售回答]\n理解您的困惑。让我们来仔细看看您的需求，来比较两种型号的性能参数以及它们各自的优势。这样，您可以依据您最关心的特点做出最好的选择。')

In [38]:
len(docs)

100

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [39]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
import os

BASE_URL = os.environ.get("BASE_URL")
API_KEY = os.environ.get("API_KEY")

db = FAISS.from_documents(docs, OpenAIEmbeddings(base_url=BASE_URL, api_key=API_KEY))

In [40]:
query = "我担心这款车的燃油效率"

In [41]:
answer_list = db.similarity_search(query)

In [42]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题]
这辆车的油耗怎么样？
[销售回答]
这款车的燃油效率是其一大亮点，它的油耗在同级别车型中相当优秀。这不仅能减少您的加油次数，长期来看还能为您节省相当一部分开支。

[客户问题]
油耗和养车成本是多少？
[销售回答]
这款车型在其类别中拥有极为竞争力的燃油效率，每公升可以行驶X公里。定期保养和一些基本的维护可以使您的续航里程达到最大化，从而进一步降低了养车成本。我们也提供透明的维护价格，没有隐藏费用。

[客户问题]
我想要一款低油耗的车型，你们有什么可以推荐的？
[销售回答]
我们有几款车型都是行业内燃油效率的佼佼者。我可以根据您的驾驶习惯和旅程需求为您推荐合适的低油耗车型。

[客户问题]
这辆车的碳排放量是多少？
[销售回答]
这款车采用了先进的环保技术，以减少排放和提高燃油效率。具体的碳排放量取决于您选择的车型和配置，但整体而言，它符合严格的环保标准。


In [43]:
db.save_local("real_car_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [44]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 5})

In [45]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020672DBC100>, search_kwargs={'k': 5})

In [46]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

D:\anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[客户问题]
这辆车的油耗怎么样？
[销售回答]
这款车的燃油效率是其一大亮点，它的油耗在同级别车型中相当优秀。这不仅能减少您的加油次数，长期来看还能为您节省相当一部分开支。

[客户问题]
油耗和养车成本是多少？
[销售回答]
这款车型在其类别中拥有极为竞争力的燃油效率，每公升可以行驶X公里。定期保养和一些基本的维护可以使您的续航里程达到最大化，从而进一步降低了养车成本。我们也提供透明的维护价格，没有隐藏费用。

[客户问题]
我想要一款低油耗的车型，你们有什么可以推荐的？
[销售回答]
我们有几款车型都是行业内燃油效率的佼佼者。我可以根据您的驾驶习惯和旅程需求为您推荐合适的低油耗车型。

[客户问题]
这辆车的碳排放量是多少？
[销售回答]
这款车采用了先进的环保技术，以减少排放和提高燃油效率。具体的碳排放量取决于您选择的车型和配置，但整体而言，它符合严格的环保标准。

[客户问题]
这款车油箱的容量是多少？
[销售回答]
这款车型的油箱容量为X升，这样的设计能够满足长途旅行的需求，减少加油的次数，让长途驾驶变得更加便捷和经济。


In [47]:
docs = topK_retriever.get_relevant_documents("我听说这个品牌的售后服务不是很好？")

In [48]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题]
贵公司提供哪些售后服务？
[销售回答]
我们提供全面的售后服务包括定期免费检查、维修服务和24小时道路救援，确保您获得最佳的车主体验和完全的心安理得。

[客户问题]
这款车的维修成本高吗？
[销售回答]
这款车的整体维护成本在行业中处于较低水平。它的可靠性和延长的保修期限意味着您将享受到持久的高性能和低维修费用。

[客户问题]
如果我不喜欢这款车，我能退还或换别的车型吗？
[销售回答]
我们有明确的满意保证政策。如果您在购车后的一定时间内对车辆不满意，我们提供退货或换车服务，具体条款我们会在购买时详细说明。

[客户问题]
你们能提供何种质量保证？
[销售回答]
我们对所有出售的车辆都非常有信心，并提供全面的质量保证。除了长期的保修之外，我们也提供了满意度保证，如果车辆在特定期限内有任何问题，您可以选择退换。

[客户问题]
新车有折旧吗？
[销售回答]
任何新车从购买开始都会有一定的折旧，但是我们所销售的车型以保值率高而闻名。我们的品牌也经常获得最佳残值奖项，这意味着您未来出售时可以得到更好的回报。


#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [49]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [50]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题]
这辆车的油耗怎么样？
[销售回答]
这款车的燃油效率是其一大亮点，它的油耗在同级别车型中相当优秀。这不仅能减少您的加油次数，长期来看还能为您节省相当一部分开支。

[客户问题]
油耗和养车成本是多少？
[销售回答]
这款车型在其类别中拥有极为竞争力的燃油效率，每公升可以行驶X公里。定期保养和一些基本的维护可以使您的续航里程达到最大化，从而进一步降低了养车成本。我们也提供透明的维护价格，没有隐藏费用。

[客户问题]
我想要一款低油耗的车型，你们有什么可以推荐的？
[销售回答]
我们有几款车型都是行业内燃油效率的佼佼者。我可以根据您的驾驶习惯和旅程需求为您推荐合适的低油耗车型。

[客户问题]
这辆车的碳排放量是多少？
[销售回答]
这款车采用了先进的环保技术，以减少排放和提高燃油效率。具体的碳排放量取决于您选择的车型和配置，但整体而言，它符合严格的环保标准。


### 提取向量数据库中的`销售回答`

In [51]:
docs = retriever.get_relevant_documents(query)

In [71]:
docs[0].page_content

'[客户问题]\n这辆车的油耗怎么样？\n[销售回答]\n这款车的燃油效率是其一大亮点，它的油耗在同级别车型中相当优秀。这不仅能减少您的加油次数，长期来看还能为您节省相当一部分开支。'

In [100]:
docs[0].page_content.split("[销售回答]")

['[客户问题]\n这辆车的油耗怎么样？\n',
 '\n这款车的燃油效率是其一大亮点，它的油耗在同级别车型中相当优秀。这不仅能减少您的加油次数，长期来看还能为您节省相当一部分开支。']

In [103]:
ans = docs[0].page_content.split("[销售回答]")[-1].lstrip('\n')

In [104]:
ans

'这款车的燃油效率是其一大亮点，它的油耗在同级别车型中相当优秀。这不仅能减少您的加油次数，长期来看还能为您节省相当一部分开支。'

#### 尝试各种问题

In [116]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答]")[-1].lstrip('\n') for doc in docs]

    return ans_list

In [117]:
query = "这个价格对我来说有点高"

print(sales(query))

[]


D:\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [118]:
print(sales(query, 0.6))

['由于这款车的安全评级非常高，保险公司通常会给予更优惠的保费。我可以帮您联系我们的保险合作伙伴，为您提供一个具体的保费报价。', '这款车的整体维护成本在行业中处于较低水平。它的可靠性和延长的保修期限意味着您将享受到持久的高性能和低维修费用。', '我们的车型设计时就考虑到了维修的便捷性和成本效益。我们还提供各种维修保养套餐，可以帮助您更好地管理维修成本。', '很高兴您提到这一点，因为这款车的保值率是它的一大卖点。它的设计经典时尚，技术先进，因此无论现在还是几年后出售，都能保持良好的市场价值。']


In [119]:
query = "我听说维修成本很高昂"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

score:0.8 ans: ['我们的车型设计时就考虑到了维修的便捷性和成本效益。我们还提供各种维修保养套餐，可以帮助您更好地管理维修成本。', '这款车的整体维护成本在行业中处于较低水平。它的可靠性和延长的保修期限意味着您将享受到持久的高性能和低维修费用。']

score:0.75 ans: ['我们的车型设计时就考虑到了维修的便捷性和成本效益。我们还提供各种维修保养套餐，可以帮助您更好地管理维修成本。', '这款车的整体维护成本在行业中处于较低水平。它的可靠性和延长的保修期限意味着您将享受到持久的高性能和低维修费用。']

score:0.5 ans: ['我们的车型设计时就考虑到了维修的便捷性和成本效益。我们还提供各种维修保养套餐，可以帮助您更好地管理维修成本。', '这款车的整体维护成本在行业中处于较低水平。它的可靠性和延长的保修期限意味着您将享受到持久的高性能和低维修费用。', '由于这款车的安全评级非常高，保险公司通常会给予更优惠的保费。我可以帮您联系我们的保险合作伙伴，为您提供一个具体的保费报价。', '这款车型在其类别中拥有极为竞争力的燃油效率，每公升可以行驶X公里。定期保养和一些基本的维护可以使您的续航里程达到最大化，从而进一步降低了养车成本。我们也提供透明的维护价格，没有隐藏费用。']


#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [120]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
import os

BASE_URL = os.environ.get("BASE_URL")
API_KEY = os.environ.get("API_KEY")

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5, base_url=BASE_URL, api_key=API_KEY)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [121]:
qa_chain({"query": "我担心新车一旦驶出展厅会立即贬值"})

D:\anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'query': '我担心新车一旦驶出展厅会立即贬值',
 'result': '我理解您的担忧，确实任何新车一旦驶出展厅都会有一定程度的贬值，这是汽车市场的普遍现象。不过，选择一款设计考虑到未来趋势的车型，可以在一定程度上减缓贬值速度。此外，我们提供的升级服务也能够帮助您的车辆保持现代化，这样可以在未来保持较高的转售价值。选择一款技术先进、风格经典的车型，长期来看是一个明智的决定。'}

In [122]:
qa_chain({"query": "最贵的车是哪一辆?"})

D:\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '最贵的车是哪一辆?',
 'result': '截至我知识更新的时间点（2023年），全球最贵的车辆可能会随着时间而变化，但历史上，一些车辆因其稀有性、品牌、历史和工艺而著称于世，其价值可能高达数百万美元。例如，法拉利250 GTO、劳斯莱斯Sweptail和布加迪La Voiture Noire等车型都是知名的高价车辆，其中布加迪La Voiture Noire的价格超过了1800万美元，成为了当时公开报道的最贵新车。\n\n然而，如果您是指最新的数据或者某个特定的拍卖记录，我可能无法提供最新的信息。汽车的价值可能会因为私人交易、拍卖结果或者制造商发布的新车而发生变化。如果您需要最新的信息，请查阅最新的新闻报道或者汽车拍卖结果。'}

In [123]:
print(sales("最贵的车是哪一辆?"))

[]


D:\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


## 加载 FAISS 向量数据库已有结果

In [124]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import os

BASE_URL = os.environ.get("BASE_URL")
API_KEY = os.environ.get("API_KEY")

db = FAISS.load_local("real_car_sale", OpenAIEmbeddings(base_url=BASE_URL, api_key=API_KEY), allow_dangerous_deserialization='True')

In [125]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5, base_url=BASE_URL, api_key=API_KEY)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

D:\anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [126]:
qa_chain({"query": "哪种车是最推荐的？"})

{'query': '哪种车是最推荐的？',
 'result': '对不起，我无法给出具体的车型推荐，因为这需要根据您的具体需求、预算和驾驶环境来决定。但是，根据上面的信息，如果您有较大的家庭，SUV和多功能车(MPV)系列可能是个好选择。'}

In [127]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [128]:
qa_chain({"query": "哪种车是最推荐的？"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '哪种车是最推荐的？',
 'result': '对不起，我不能直接告诉你哪种车是最推荐的，因为这取决于你的具体需求，包括你的预算、驾驶环境、旅程需求等。销售人员会根据这些信息向你推荐几款最适合你的车型。'}

In [129]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [130]:
result = qa_chain({"query": "哪种车是最推荐的？"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


In [131]:
result

{'query': '哪种车是最推荐的？',
 'result': '对不起，我无法提供具体的推荐，因为这需要根据您的个人需求和预算来决定。例如，如果您需要大量的空间和乘坐人数多，可能会推荐SUV或MPV。如果您关心燃油效率，可能会推荐一些低油耗车型。最好的建议是与销售人员进行详细的讨论，他们可以根据您的具体需求为您推荐最合适的车型。',
 'source_documents': [Document(page_content='[客户问题]\n我该怎么选择合适的车款？\n[销售回答]\n首先，我们将讨论您的日常用车需求，包括乘客数量、驾驶环境和预算。然后，我会根据这些信息向您推荐几款最适合您的车型，帮助您做出最佳选择。'),
  Document(page_content='[客户问题]\n我想要一款低油耗的车型，你们有什么可以推荐的？\n[销售回答]\n我们有几款车型都是行业内燃油效率的佼佼者。我可以根据您的驾驶习惯和旅程需求为您推荐合适的低油耗车型。'),
  Document(page_content='[客户问题]\n我们有一大家人，你推荐哪款车？\n[销售回答]\n考虑到您家庭的需求，我建议您了解我们的SUV和多功能车(MPV)系列。这些车型提供充足的空间、高安全标准以及适合家用的娱乐系统选项。')]}